#### Load Libraries and Define Functions

In [1]:
if True:
    from julia.api import Julia
    jl = Julia(compiled_modules=False)

import julia; julia.install(quiet=True)
from julia import Main

import numpy     as np
import param
import holoviews as hv; hv.extension("bokeh", logo=False)
import panel as pn;     pn.extension("perspective", "katex")

In [2]:
%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [3]:
%%julia
using Pkg; Pkg.activate("../GenLinAlgProblems")
using GenLinAlgProblems, LinearAlgebra, RowEchelon, Printf, Latexify, LaTeXStrings, Random, SymPy

  Activating project at `C:\Users\jeff\NOTEBOOKS\elementary-linear-algebra\GenLinAlgProblems`


In [4]:
%%julia
function principal_angles_form_basis_matrices(U::AbstractMatrix, V::AbstractMatrix)
    S = svd(U'V).S
    clamped = clamp.(S, -1, 1)
    return acosd.(clamped)
end;

In [5]:
%%html
<script src="https://cdnjs.cloudflare.com/ajax/libs/paper.js/0.12.18/paper-full.min.js"></script>

In [6]:
class Gr_1_2_Viewer(pn.viewable.Viewer, pn.custom.JSComponent):
    angle = param.Number(default=45, bounds=(0, 180), label="Angle (degrees)")

    _esm = """
    export function render({ model, el }) {
        const width  = 300;
        const height = 270;
        const xrange = [-1.5, 1.5];
        const yrange = [-0.3, 2.0];

        const canvas = document.createElement('canvas');
        canvas.width = width;
        canvas.height = height;
        canvas.style.border = "1px solid black";
        el.appendChild(canvas);

        const paperScope = new paper.PaperScope();
        paperScope.setup(canvas);

        const worldToScreen = (point) => {
            const x = (point.x - xrange[0]) / (xrange[1] - xrange[0]) * width;
            const y = height - (point.y - yrange[0]) / (yrange[1] - yrange[0]) * height;
            return new paperScope.Point(x, y);
        };

        function drawArrow(start, end, color, label, dashed = false, labelOffset = null) {
            const startPoint = worldToScreen(start);
            const endPoint = worldToScreen(end);

            const line = new paperScope.Path.Line({
                from: new paperScope.Point(startPoint.x, startPoint.y),
                to: new paperScope.Point(endPoint.x, endPoint.y),
                strokeColor: color,
                strokeWidth: 2,
                dashArray: dashed ? [5, 5] : null,
            });

            const arrowSize = 10;
            const direction = new paperScope.Point(endPoint.x - startPoint.x, endPoint.y - startPoint.y).normalize();

            const arrowHead = new paperScope.Path({
                segments: [
                    new paperScope.Point(endPoint.x, endPoint.y),
                    new paperScope.Point(
                        endPoint.x - direction.x * arrowSize + direction.y * arrowSize / 2,
                        endPoint.y - direction.y * arrowSize - direction.x * arrowSize / 2
                    ),
                    new paperScope.Point(
                        endPoint.x - direction.x * arrowSize - direction.y * arrowSize / 2,
                        endPoint.y - direction.y * arrowSize + direction.x * arrowSize / 2
                    )
                ],
                closed: true,
                fillColor: color,
            });

            if (label) {
                let labelPoint = new paperScope.Point(endPoint.x, endPoint.y);
                if (labelOffset) {
                    labelPoint = labelPoint.add(new paperScope.Point(labelOffset.x, labelOffset.y));
                } else {
                    const perp = new paperScope.Point(-direction.y, direction.x).normalize().multiply(15);
                    labelPoint = labelPoint.add(perp);
                }
                new paperScope.PointText({
                    point: labelPoint,
                    content: label,
                    fillColor: color,
                    fontSize: 12,
                    justification: 'center',
                });
            }
        }

        function redraw() {
            paperScope.project.clear();

            const origin = { x: 0, y: 0 };
            const angleRad = model.angle * Math.PI / 180;

            const u = { x: Math.cos(angleRad), y: Math.sin(angleRad) };
            const du = { x: -Math.sin(angleRad), y: Math.cos(angleRad) };

            // Draw unit vector u
            drawArrow(origin, u, 'blue', 'u(θ)');

            // Draw tangent vector du/dθ starting from tip of u
            const tip = { x: u.x + 0.4 * du.x, y: u.y + 0.4 * du.y };
            drawArrow(u, tip, 'red', "du/dθ", true);

            // Draw arc for angle θ
            const arcRadius = 0.3;
            const arcCenter = worldToScreen(origin);
            const midAngle = angleRad / 2;
            const mid = {
                x: arcRadius * Math.cos(midAngle),
                y: arcRadius * Math.sin(midAngle)
            };

            const arc = new paperScope.Path.Arc({
                from: worldToScreen({ x: arcRadius, y: 0 }),
                through: worldToScreen(mid),
                to: worldToScreen({ x: arcRadius * Math.cos(angleRad), y: arcRadius * Math.sin(angleRad) }),
                strokeColor: 'black',
                strokeWidth: 2
            });

            // Add angle label
            const labelOffset = { x: 10, y: -10 };
            const labelPoint = worldToScreen(mid);
            new paperScope.PointText({
                point: labelPoint.add(new paperScope.Point(labelOffset.x, labelOffset.y)),
                content: `${model.angle.toFixed(1)}°`,
                fillColor: 'black',
                fontSize: 14,
                justification: 'center'
            });
        }

        redraw();
        model.on("angle", () => redraw());
    }
    """

    def __init__(self, **params):
        super().__init__(**params)
        self.latex_pane = pn.pane.LaTeX(width=300, renderer='katex')
        self.slider = pn.widgets.FloatSlider.from_param(self.param.angle, width=300)
        self.param.watch(self._update_latex, 'angle')
        self._update_latex()

    def _update_latex(self, *_):
        angle_rad = np.radians(self.angle)
        u = np.array([np.cos(angle_rad), np.sin(angle_rad)])
        du = np.array([-np.sin(angle_rad), np.cos(angle_rad)])
        self.latex_pane.object = (
            "<div style='font-size:18px;font-family:monospace;'>"
            r"$\Large \begin{aligned}"
            r"u(\theta) &= \begin{bmatrix} "
            f"{u[0]:.2f} \\\\ {u[1]:.2f} "
            r"\end{bmatrix} \\[6pt]"
            r"\frac{d}{d\theta}u &= \begin{bmatrix} "
            f"{du[0]:.2f} \\\\ {du[1]:.2f} "
            r"\end{bmatrix}"
            r"\end{aligned}$"
            "</div>"
        )

    def __panel__(self):
        return pn.Row(
            self,
            pn.Spacer(width=30),
            pn.Column(
                pn.Spacer(height=20),
                self.latex_pane,
                pn.Spacer(height=20),
                self.slider,
            )
        )

gr_1_2 = Gr_1_2_Viewer()
pn.panel(gr_1_2).servable("Grassmannian Viewer")

Row
    [0] Gr_1_2_Viewer()
    [1] Spacer(width=30)
    [2] Column
        [0] Spacer(height=20)
        [1] LaTeX(str, renderer='katex', width=300)
        [2] Spacer(height=20)
        [3] FloatSlider(end=180, name='Angle (degrees)', value=45, width=300)

#### 

<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">The Grassmannian</strong>
</div></div>

# 1. Introduction of the Grassmannian

In previous notebooks starting with [**HyperplanesPrincipalAngles.ipynb**](HyperplanesPrincipalAngles.ipynb),<br>
we compared subspaces using principal angles and defined distance metrics based on them.

But all of these comparisons live inside a larger structure:<br>
$\qquad$ **the space of all $k$-dimensional subspaces of $\mathbb{R}^n$.**

This space — curved, nonlinear, and rich in structure —
is the **Grassmannian manifold**, denoted $\mathrm{Gr}(k, n)$.

In this notebook, we introduce the Grassmannian from a geometric perspective:
- Subspaces as points on a manifold
- Tangent spaces and local structure
- Intrinsic distances and geodesics

This sets the stage for interpolation, optimization, and clustering
directly on the space of subspaces.

## 1.1 Definition

<div style="background-color:#F2F5A9;color:black;padding-bottom:0.05cm;">

**Def:** The **Grassmannian** $\mathrm{Gr}(k, n)$ is the set of all
$k$-dimensional linear subspaces of $\mathbb{R}^n$.

$\qquad
\mathrm{Gr}(k, n) = \{ \mathcal{U} \subset \mathbb{R}^n \mid \dim(\mathcal{U}) = k \}
$
</div>

Each element of $\mathrm{Gr}(k, n)$ is a $k$-dimensional subspace of $\mathbb{R}^n$:<br>
a geometric object, defined independently of any specific basis.

We will treat $\mathrm{Gr}(k, n)$ as a geometric space,
and explore its structure in the sections that follow.
<div>

**Examples:**
- $\mathrm{Gr}(1, n)$: all 1-dimensional subspaces of $\mathbb{R}^n$.<br>
  Each element is a line through the origin
- $\mathrm{Gr}(n-1, n)$: hyperplanes of dimension $n-1$ in $\mathbb{R}^n$
- $\mathrm{Gr}(n, n)$: a single point — the whole space $\mathbb{R}^n$
-  $\mathrm{Gr}(k, n)$ with $1 < k < n-1$: the general case.
---
</div>

## 1.2 Subspaces as Points

Each element of $\mathrm{Gr}(k, n)$ is a $k$-dimensional subspace — an abstract geometric object.<br>
But to work with subspaces in practice, we **represent them using coordinate systems.**

---
Common representations of a subspace:
- **Orthonormal basis matrix**: $U \in \mathbb{R}^{n \times k}$, where columns of $U$ form an orthonormal basis for the subspace
- **Projection operator**: $P_u = U U^T\;$ maps $\mathbb{R}^n$ onto the subspace
- **Equivalence class**: all matrices $U$ with the same column space $U \sim U Q$ for any $Q \in \mathrm{O}(k)$ (orthogonal change of basis)<br>
 here $\mathrm{O}(k)$ is the group of $k \times k$ orthogonal matrices: $Q^\top Q = I$ under matrix multiplication.
<details style="padding-left:1cm;">
Let $U, \tilde{U} \in \mathbb{R}^{n \times k}$ be orthonormal basis matrices for the same subspace $\mathcal{U} \subset \mathbb{R}^n$.<br>
Since $\operatorname{im}(\tilde{U}) = \operatorname{im}(U)$,
each column of $\tilde{U}$ is a linear combination of the columns of $U$,<br>
$\qquad$ i.e., $\tilde{U} = U Q$ for some $Q \in \mathbb{R}^{k \times k}$.<br>
Orthonormality of $\tilde{U}$ gives<br>
$\qquad I = \tilde{U}^T \tilde{U} = Q^T U^T U Q = Q^\top Q$<br>
so $Q \in \mathrm{O}(k)$.
</details>

Each of these describes the same subspace, but encodes different levels of structure.

**Important Remark:** the projection operator is **independent of any chosen basis.**<br>
$\qquad$ given orthogonal basis matrices $U$ and $V$, they span the same subspace iff $U U^T = V V^T$.

---

These representations allow us to embed $\mathrm{Gr}(k, n)$ into familiar matrix spaces — for computation and visualization.

#### Example: Representing a 3-dimensional subspace of $\mathbb{R}^4$

Consider a randomly chosen 3-plane $\mathcal{U} \subset \mathbb{R}^4$ specified by some basis.

We show each of the representations of this 3-pane
- An orthonormal basis $U$
- Another basis $\tilde{U} = U Q$ with $Q \in \mathrm{O}(3)$
- The projection matrix $P_u = U U^T$

and verify that both $U$ and $\tilde{U}$ yield the same projector, i.e., the projector does not depend on the chosen basis.

In [16]:
%%julia
function ex_1()
    # Step 1: Generate random 3D subspace in ℝ⁴
    A = gen_full_col_rank_matrix(4, 3)        # basis matrix for a 3-plane
    U, _ = qr(A);                             # Orthonormal basis matrix for this 3-plane
    U    = Matrix(U)
    # Step 2: Rotate the basis using Q ∈ O(3)
    Q = qr(gen_full_col_rank_matrix(3, 3)).Q  # Random orthogonal matrix
    Q = Matrix(Q)
    Ũ = U * Q                                 # Another 3-plane

    # Step 3: Compute projection matrices
    P = U * U'           # Projection onto im(U)
    P̃ = Ũ * Ũ'           # Should be the same

    # Output
    display( py_show(  "Original basis matrix A =", A,L"\qquad \text{a 3-plane in } \mathbb{R}^4",  number_formatter=x->round_value(x,2)));
    display( py_show(  "Representations:"));
    display( py_show( L"\qquad U = ", U, L"\qquad", "an orthonormal basis matrix", number_formatter=x->round_value(x,2)))
    display( py_show( L"\qquad\tilde{U} = ", Ũ, L"\qquad", "a different orthonormal basis matrix for the same 3-plane", number_formatter=x->round_value(x,2)))
    display( py_show( L"\qquad P = ", P, L"\qquad", "the projection matrix computed using U", number_formatter=x->round_value(x,2)))
    display( py_show( L"\qquad \tilde{P} =", P̃, L"\qquad", "the projection matrix computed using ", L"\tilde{U}", number_formatter=x->round_value(x,2)))
    display(py_show( "Projection matrices are independent of the basis used:"))
@show P ≈ P̃;
end
Random.seed!(42)
ex_1();

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

P ≈ P̃ = true

## 1.3 Dimension of the Grassmannian

How many degrees of freedom are needed to specify a point in $\mathrm{Gr}(k, n)$? Each point is a $k$-dimensional subspace of $\mathbb{R}^n$.

---
We begin with a matrix $U \in \mathbb{R}^{n \times k}$
whose columns form an orthonormal basis.
- This has $n k$ entries
- The orthonormality condition $U^T U = I$ imposes $\frac{k(k + 1)}{2}$ constraints.

But this still includes different orthonormal bases for the same subspace.<br>
$\qquad$ To remove this redundancy, we quotient by right multiplication with $\mathrm{O}(k)$,
which itself has dimension $\frac{k(k - 1)}{2}$.

---

Total dimension:

$\qquad
\dim \mathrm{Gr}(k, n) = n k - \frac{k(k + 1)}{2} - \frac{k(k - 1)}{2} = k(n - k)
$

This matches our intuition:

- Choosing $k$ independent directions in $n$-space
- Modulo rotations within the subspace itself

---
<details style="padding-left:1cm;">
<summary><strong>Alternate intuition: build a $k$-plane step by step</strong></summary>

To define a $k$-dimensional subspace of $\mathbb{R}^n$:

- First vector: $\qquad n - 1$ degrees of freedom (unit sphere in $\mathbb{R}^n$)
- Second: $\qquad\quad n - 2$ (orthogonal to the first)
- …
- $k$-th: $\qquad\qquad n - k$ (orthogonal to the previous $k - 1$)

Total: $k n - \frac{k(k + 1)}{2}$

We then divide out rotations within the subspace
(given by $\mathrm{O}(k)$ of dimension $\frac{k(k - 1)}{2}$)

Final count: $k(n - k)$ degrees of freedom.
</details>


# 2. Geometry on the Grassmannian

## 2.1. Geometric Structure of the Grassmannian

The Grassmannian $\mathrm{Gr}(k, n)$ is the set of all $k$-dimensional linear subspaces of $\mathbb{R}^n$.

Each element is itself a subspace, i.e., a set of vectors closed under linear combinations<br>
but $\mathrm{Gr}(k, n)$ as a whole is **not a vector space.**

We do not define operations like addition or scalar multiplication between subspaces.<br>
Instead, the Grassmannian has a **geometric structure**:<br>
we can measure **distances between subspaces,** define **smooth paths,** and study how they vary.

To see this, consider the following two examples.

### 2.1.1 Example 1: $\mathrm{Gr}(1, 2)$ — Lines through the origin

The set of all 1D subspaces in $\mathbb{R}^2$ consists of all lines through the origin.

Each such subspace is determined by a direction, that is<br>
by a unit vector $u$, **up to sign** (since $u$ and $-u$ span the same line).

Using polar coordinates, these directions are specified by angles $\theta$, so the space of directions forms a circle<br>
where opposite points are considered the same.

This makes $\mathrm{Gr}(1, 2)$ a **1-dimensional manifold**:
- It has one degree of freedom (the angle of the line),
- It has no boundary (the endpoints $\theta = 0$ and $\theta = \pi$ represent the same line),
- And if we zoom in on any point (i.e., any line,) the nearby subspaces can be described by a small change in angle<br>
just like moving left or right along a line.

This is the simplest nontrivial Grassmannian.

<details>
<summary><strong>What does it mean to be a smooth manifold?</strong></summary>

A smooth manifold is a space that **locally resembles** $\mathbb{R}^d$ — meaning:

- Near any point, you can assign coordinates that behave like real numbers, (e.g., $\theta$ in the current example
- You can take derivatives of functions defined on the space,
- And the space has no “edges,” corners, or discrete jumps.

So although the Grassmannian isn't a vector space, it has a **differentiable structure** that lets us do calculus on subspaces.
</details>

### 2.1.2 Example 2: Local coordinates via projection matrices

To understand how the Grassmannian behaves near a given subspace, we examine a **concrete example**.

Let $\mathcal{U}$ be a 2D subspace of $\mathbb{R}^3$, with orthonormal basis matrix $U \in \mathbb{R}^{3 \times 2}$.<br>
We define a second subspace $\mathcal{V}$ by **rotating one of $U$'s basis vectors slightly**, producing a new matrix $V$.

This gives:

$\qquad P_u = U U^T, \quad P_v = V V^T, \quad \Delta = P_v - P_u$

The matrix $\Delta$ is a **symmetric perturbation** that tells us how $\mathcal{V}$ differs from $\mathcal{U}$.

In [5]:
%%julia
function ex_2_1_2()
    # Base subspace: 2D plane in R³
    A = gen_full_col_rank_matrix(3, 2)        # basis matrix for a 3-plane
    U = Matrix(qr(A).Q)                       # Orthonormal basis matrix for this 3-plane

    # Slightly rotate one column to make a nearby subspace
    θ = 5 * (π / 180)  # 5 degree rotation
    R = [cos(θ) -sin(θ); sin(θ) cos(θ)]
    V = copy(U)
    V[1:2, 1] = R * U[1:2, 1]    # rotate first column slightly
    V = Matrix(qr(V).Q)  # re-orthonormalize

    # Projectors
    P_u     = U * U'
    P_v     = V * V'
    Δ       = P_v - P_u
    angles  = principal_angles_form_basis_matrices(U, V)'

    display(py_show( "Principal Angles ", set( angles...,  number_formatter=x->round_value(x,2))))
    display(py_show( L"\Vert P_v - P_u \Vert = ", norm(Δ),  number_formatter=x->round_value(x,2)))
end
Random.seed!(515)
ex_2_1_2()

<IPython.core.display.Latex object>

P ≈ P̃ = true


<IPython.core.display.Latex object>

## 2.2 Distance on the Grassmannian

We previously defined several distance metrics between subspaces based on their principal angles:
- **Spectral distance**: $\quad \sin \theta_1$
- **Chordal (Frobenius) distance**: $\quad \left( \sum_{i=1}^k \sin^2 \theta_i \right)^{1/2}$
- **Geodesic distance**: $\quad \left( \sum_{i=1}^k \theta_i^2 \right)^{1/2}$

---

Each of these distances satisfies the axioms of a metric on $\mathrm{Gr}(k, n)$.

The geodesic distance is **intrinsic**:
it measures the shortest path between two subspaces *on the manifold*.

The others — spectral, chordal — are **extrinsic**:<br>
they measure distances in the ambient matrix space
via projection operators or principal angles.

<details>
<summary><strong>What do we mean by manifold, intrinsic, and extrinsic?</strong></summary>

<br>

**Manifold:** A manifold is a space that may be curved globally,<br>
$\qquad$ but looks flat locally — like a surface where we can measure distances and angles.

The Grassmannian $\mathrm{Gr}(k, n)$ is a manifold: each point is a subspace, and nearby subspaces vary smoothly.

---

**Intrinsic vs. Extrinsic Distances.**

- An **intrinsic** distance measures the shortest path *on* the manifold —
  using only information from within the space.

- An **extrinsic** distance measures separation *from outside* —
  for example, using coordinates, projections, or embeddings in matrix space.
</details>

---

All three metrics arise from the same set of principal angles but emphasize different aspects of subspace separation.

## 2.3 Geodesics and Interpolation

**Example:**
* $\mathrm{Gr}(1, n)$ has dimension $n - 1\;\;$  (a unit direction in $\mathbb{R}^n$)
* $\mathrm{Gr}(n-1, n)\;\;$ also has dimension $n - 1$
* $\mathrm{Gr}(k, n)$ and $\mathrm{Gr}(n - k, n)\;$ have the same dimension

To move smoothly between two subspaces, we want a curve that stays entirely inside the Grassmannian<br>
and connects them **with minimal distance**, i.e., a geodesic.

---

Each principal angle $\theta_i$ defines a 2D plane spanned by the corresponding principal vectors $\tilde{u}_i$ and $\tilde{v}_i$.<br>
To interpolate between subspaces, we rotate each pair $(\tilde{u}_i, \tilde{v}_i)$ in its principal plane

$\qquad
\tilde{u}_i(t) = \tilde{u}_i \cos(\theta_i t) + \tilde{w}_i \sin(\theta_i t)
$

Here, $\tilde{w}_i$ is a unit vector orthogonal to $\tilde{u}_i$ that lies in the plane spanned by $\tilde{u}_i$ and $\tilde{v}_i$.<br>
It is chosen so that the rotation from $\tilde{u}_i$ to $\tilde{v}_i$ is traced out as $t$ moves from 0 to 1, i.e.,

$\qquad
\tilde{w}_i = \frac{1}{\sin(\theta_i)} \left( \tilde{v}_i - \cos(\theta_i)\, \tilde{u}_i \right)
\quad \text{(if } \theta_i \ne 0 \text{)}
$

This ensures $\tilde{u}_i^\top \tilde{w}_i = 0,$ $\Vert \tilde{w}_i\Vert = 1$ and $\tilde{u}_i(1) = \tilde{v}_i$
____

Stacking the vectors into matrices gives the geodesic path

$\qquad
U(t) = \tilde{U} \cos(\Theta t) + \tilde{W} \sin(\Theta t)
$

where

- $\tilde{U} = U L$, $\tilde{V} = V R$ are the principal vectors from the SVD of $U^\top V = L \cos(\Theta) R^\top$
- $\tilde{W} = [\tilde{w}_1 \ \dots\ \tilde{w}_r]$
- $\Theta = \operatorname{diag}(\theta_1, \dots, \theta_r)$

This construction guarantees that:

- $U(0) = \tilde{U}$
- $U(1) = \tilde{V}$
- Each $U(t)$ spans a subspace lying along the geodesic from $\mathcal{U}$ to $\mathcal{V}$ in the Grassmannian.

<div>
This path rotates each direction separately
and yields the shortest possible curve between subspaces<br>
under the geodesic metric on $\mathrm{Gr}(k, n)$.

<details>
<summary><strong>Why this path stays in the Grassmannian — and why it's a geodesic</strong></summary>

* **Why it stays in the Grassmannian.**

<div style="padding-left:1cm;">

Each pair $(\tilde{u}_i, \tilde{w}_i)$ spans a 2D plane.

The interpolation<br>
$\qquad
\tilde{u}_i(t) = \tilde{u}_i \cos(\theta_i t) + \tilde{w}_i \sin(\theta_i t)
$

is a rotation in that plane. Since the rotations are independent and orthogonal,<br>
the full system $U(t)$ remains orthonormal for all $t$.

Thus, $\operatorname{im}(U(t))$ is a valid $k$-dimensional subspace —i.e., a point on $\mathrm{Gr}(k, n)$.

---
</div>

* **Why it defines a geodesic.**

<div style="padding-left:1cm;">

This construction yields the shortest path between $\mathcal{U}$ and $\mathcal{V}$<br>
under the natural distance induced by the principal angles:

$\qquad
\mathrm{dist}(\mathcal{U}, \mathcal{V}) = \left( \sum \theta_i^2 \right)^{1/2}
$

While a full justification requires tools from differential geometry,<br>
we take this as a fact: the formula defines a true geodesic on $\mathrm{Gr}(k, n)$.
</div>
</details>
</div>

### Example Computation

In [6]:
%%julia
# Example: Geodesic interpolation between two subspaces
function ex_2_3()
    # #################################### Two random 3D subspaces in ℝ⁴
    A = gen_full_col_rank_matrix(4, 2)
    B = gen_full_col_rank_matrix(4, 2)
    U, _ = qr(A); U = Matrix(U)          # orthonormal basis matrix
    V, _ = qr(B); V = Matrix(V)          # orthonormal basis matrix

    # #################################### Compute principal vectors and angles
    # Principal angles
    F = svd(U' * V)
    θ = acos.(clamp.(F.S, -1.0, 1.0))
    display(py_show("Principal angles for U, V: ", round.(acosd.(clamp.(F.S, -1.0, 1.0))', digits=1) ))
    Θ = Diagonal(θ)

    # Aligned principal vector systems
    Ũ = U * F.U
    Ṽ = V * F.V

    # Compute W̃
    W̃ = Matrix{Float64}(undef, size(U, 1), 0)
    for i in 1:length(θ)
        if isapprox(θ[i], 0.0; atol=1e-10)
            continue  # No rotation needed
        end
        w = (Ṽ[:, i] - cos(θ[i]) * Ũ[:, i]) / sin(θ[i])
        w = normalize(w)  # Ensure unit length
        W̃ = hcat(W̃, w)
    end

    if size(W̃, 2) == 0
        W̃ = zeros(size(Ũ))  # zero matrix of shape (n × k)
    end
    # -------------------------------------------------------------------
    U_interp(t) = Ũ * Diagonal(cos.(θ .* t)) + W̃ * Diagonal(sin.(θ .* t))

    # Sample geodesic path
    ts               = 0:0.05:1.0
    interpolates     = Matrix{Float64}[]
    angle_evolution  = Vector{Vector{Float64}}()

    for t in ts
        Ut = U_interp(t)
        Ut = Matrix(qr(Ut).Q)           # reorthogonalize for better numerical behavior
        push!(interpolates, Ut)

        # Compute principal angles between U and Ut
        Ft = svd(U' * Ut)
        push!(angle_evolution, acosd.(clamp.(Ft.S, -1.0, 1.0)))
    end

    Ut = interpolates
    # #################################### Checks
    display(py_show( "Interpolates represented by their projector matrices: ", L"U^T U \rightarrow V^TV" ))
    display(py_show(Ut[1]*Ut[1]', L"\rightarrow", Ut[2]*Ut[2]', L"\rightarrow\quad \dots\quad \rightarrow ", Ut[end]*Ut[end]', number_formatter=x->round_value(x,2)))
    return U,V, ts, interpolates, angle_evolution
end

Random.seed!(234)
U,V,ts, Ut, principal_angles = ex_2_3();

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

#### Visualizing the Evolution of Subspaces

As a subspace moves along the geodesic,
its basis vectors rotate within their principal planes.

One way to visualize this evolution is to track how each **principal angles** change with time.

In [7]:
def plot_principal_angle_evolution(t, principal_angles):
    """
    Plots each principal angle as a function of time `t`.

    Parameters:
    - t: 1D array of time samples (e.g., Main.t)
    - principal_angles: List of arrays, each containing principal angles at each t
                        (e.g., Main.principal_angles — a list of 1D arrays)
    """

    # Convert list of vectors to 2D array: angles[i, j] = j-th angle at time t[i]
    angles = np.array(principal_angles)  # shape: (len(t), k)

    curves = []
    for j in range(angles.shape[1]):
        angle_values = angles[:, j]
        label        = f"θ{j+1}(t)"
        curves.append(hv.Curve((t, angle_values), 't', 'angle (degrees)').opts(
            show_grid=True,
            line_width=2,
            tools=['hover'],
            title="Principal Angle Evolution"
        ).relabel(label))

    return hv.Overlay(curves).opts( legend_position="top_left", height=300, width=600)
pn.Row(plot_principal_angle_evolution(Main.ts, Main.principal_angles))

Row
    [0] HoloViews(Overlay, height=300, sizing_mode='fixed', width=600)

	Distance on the Grassmannian	Subspace distances (e.g., geodesic, chordal) become true metrics on 
G
r
(
𝑘
,
𝑛
)
Gr(k,n). Principal angles define intrinsic distances. Clarify: this generalizes previous metrics.
5	Geodesics and Interpolation	Define shortest paths between subspaces. Give example of interpolation using principal vectors. Optional: low-dimensional illustrations (e.g. in 
G
r
(
1
,
3
)
Gr(1,3)).
6	Tangent Vectors and Optimization	Brief intro to the tangent space at a point. Mention applications: gradient descent on 
G
r
(
𝑘
,
𝑛
)
Gr(k,n), PCA, subspace learning.
7	Summary and Transition	Recap what the Grassmannian is, how distances and geometry emerge from principal angles. Link forward to applications: tracking, clustering, geodesic interpolation, etc.

# OLD

### Optional: Wedge Products and Subspaces

The wedge product $v_1 \wedge \dots \wedge v_k$ represents a $k$-dimensional volume  
spanned by the vectors $v_1, \dots, v_k$.

In later algebra, this multivector can also encode the subspace itself  
(up to scale and orientation).

See: [Wedge Product and Volume](13_WedgeProduct.ipynb)  
for a visual and geometric introduction to wedge products.


## The Grassmannian

The set of all $k$-dimensional subspaces of $\mathbb{R}^n$  
forms a smooth manifold: the **Grassmannian** $\mathrm{Gr}(k, n)$.

It provides a geometric space where subspaces become points,  
and distances between them become paths.

---

### Definition

Let $k \leq n$.  
The Grassmannian $\mathrm{Gr}(k, n)$ is the set of all $k$-dimensional linear subspaces of $\mathbb{R}^n$.

Each point on $\mathrm{Gr}(k, n)$ is a subspace,  
not a basis or a set of vectors.

---

### Representations

A subspace $U \subset \mathbb{R}^n$ can be represented by:

- An orthonormal basis matrix $U \in \mathbb{R}^{n \times k}$  
- The projection operator $P = UU^\top$  
- A point on $\mathrm{Gr}(k, n)$

Different bases → same projector → same point on the Grassmannian.

---

### Geometry

The Grassmannian has a natural distance function  
defined by the principal angles between subspaces.

- Zero distance ⇔ subspaces are identical  
- Maximal distance ⇔ subspaces are orthogonal

Subspace distances become **geodesic distances** on $\mathrm{Gr}(k, n)$.

---

### Visualization

- $\mathrm{Gr}(1, 2)$ is a circle  
- $\mathrm{Gr}(1, 3)$ is the real projective plane $\mathbb{RP}^2$  
- Higher $\mathrm{Gr}(k, n)$ are curved manifolds

We can visualize motion on the Grassmannian as  
smooth paths between subspaces.

---

### Next

We now explore **geodesics** and optimization paths on $\mathrm{Gr}(k, n)$.
